In [1]:
import getpass
import os
# QhdyPvS28vnpVY94g5YKmm58PgEhUCYb
os.environ["MISTRAL_API_KEY"] = getpass.getpass()

In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### Load the PDF

In [3]:
from langchain.document_loaders import PyPDFLoader

pdf_file_path = "ArtificialIntelligenceAct.pdf"

loader = PyPDFLoader(pdf_file_path)

# Load and process the document
docs = loader.load()

#### Create the Chunks, generate the embeddings and store them
The model we used for generating the embeddings was the ``all-MiniLM-L6-v2`` of Hugging Face.

Then we stored those embeddings in a vectorial data base (Chroma).


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

C:\Users\perei\AppData\Local\Temp\ipykernel_16604\4226516271.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8}
            )
            
prompt = hub.pull("rlm/rag-prompt")

"""Prompt template:
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

    Question: {question} 

    Context: {context} 

    Answer:
"""

print(retriever.get_relevant_documents("Hi"))

c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[Document(metadata={'page': 329, 'source': 'ArtificialIntelligenceAct.pdf'}, page_content='The Board may establish other standing or temporary sub-groups as appropriate for the \npurpose of examining specific issues. Where appropriate, representatives of the advisory \nforum referred to in Article 67 may be invited to such sub-groups or to specific meetings \nof those subgroups as observers.\n7. The Board shall be organised and operated so as to safeguard the objectivity and \nimpartiality of its activities.\n8. The Board shall be chaired by one of the representatives of the Member States. The AI \nOffice shall provide the secretariat for the Board, convene the meetings upon request of \nthe Chair, and prepare the agenda in accordance with the tasks of the Board pursuant to \nthis Regulation and its rules of procedure.\nArticle 66\nTasks of the Board\nThe Board shall advise and assist the Commission and the Member States in order to facilitate \nthe consistent and effective application

C:\Users\perei\AppData\Local\Temp\ipykernel_16604\1512139187.py:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(retriever.get_relevant_documents("Hi"))


In [6]:
print(hub.pull("langchain-ai/rag-context-recall"))

c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['documents', 'student_answer'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'rag-context-recall', 'lc_hub_commit_hash': '8b6ffd487ad2a66c0fc129ee1e3d4854336c7fd736405af0b9b1a0ffb63d51e8'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a teacher grading a quiz. \n\nYou will be given a STUDENT ANSWER and a set of FACTS. \n\nHere is the grade criteria to follow:\n(1) Look at each sentence in the STUDENT ANSWER.\n(2) Determine whether the sentence can be attributed to the FACTS.\n(3) Assign a score of 1 if the sentence can be attributed to the FACTS \n(4) Assign a score of 0 if the sentence can NOT be attributed to the FACTS \n\nScore:\nReturn the fraction: The number of sentences that can be attributed to the FACTS and got a score of 1 divided by the total number of sentences.\n\nExplain your reasoning in a step-by-step manner to ensure 

#### Modelo : MistralAI através da Langchain

In [7]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="open-mistral-7b", api_key=os.environ["MISTRAL_API_KEY"])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Pipeline de RAG
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#rag_chain.invoke("What are the main values enshrined in the AI Act, according to Union law?")
#rag_chain.invoke("What are the primary objectives of the Artificial Intelligence Act?")
rag_chain.invoke("How does the AI Act aim to regulate high-risk AI systems?")

"The AI Act aims to regulate high-risk AI systems by requiring operators to comply with its requirements by specific deadlines. For high-risk AI systems used by public authorities, this is within six years of the Regulation's entry into force. For systems already in use before this date, compliance is required if they undergo significant changes. Providers of high-risk AI systems are encouraged to comply during the transitional period. (178, 2, 3)\n\nHigh-risk AI systems are defined as those that pose a high risk of harm to health, safety, or fundamental rights, and are used in specified areas. These systems are subject to harmonized rules, prohibitions of certain AI practices, and specific requirements. (52)\n\nThe AI Act's purpose is to improve the internal market, promote human-centric and trustworthy AI, and ensure a high level of protection against harmful AI effects. It lays down harmonized rules, prohibitions, and requirements for high-risk AI systems. (1)"